In [5]:
function distributeOnQueues(queues, n)
    while n!=0 && any(queues==0)
        n -= 1
        rej_flag = true
        queue = sample(queues, Weights(1+sum(queues, dims=2)))
        for neighbour in 0:0
            ones = findall(queue == 1)
        end
    end
end


adsf

In [13]:
a = ones(2,4)
display(a)
display(sum(a,dims=2))
idx = findall(x -> x==1., a)
display(a[idx])

2×4 Matrix{Float64}:
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0

2×1 Matrix{Float64}:
 4.0
 4.0

8-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0